In [263]:
import jax.random
import matplotlib.pyplot as plt
import jax.numpy as jnp
import netket as nk
from scipy.sparse.linalg import eigsh
import numpy as np
from netket.operator.spin import sigmax,sigmaz
import itertools
    


# Parameters
ang_ = [0.0, 0.157, 0.314, 0.471, 0.628, 0.785, 0.942, 1.1, 1.257, 1.414, 1.571]
a = 0
for ang in jnp.linspace(jnp.pi/2, 0.0, 11):
    L = 6
    p = 1
    angle = ang_[a] 
    print(angle)
    M0 = np.load("mps_tensors/tensor_init_" + str(L * p) + "_angle_" + str(angle) + ".npy")
    M = np.load("mps_tensors/tensor_" + str(L * p) + "_angle_" + str(angle) + ".npy")
    Mlast = np.load("mps_tensors/tensor_last_" + str(L * p) + "_angle_" + str(angle) + ".npy")

    # Define the hi space
    hi = nk.hilbert.Spin(s=0.5, N=L)
    
    # Add the terms from the ITensor h
    # First set of terms
    h = - np.cos(angle)**2 *  sigmaz(hi, 0) @  sigmax(hi, 1)
    h -=  np.cos(angle) * np.sin(angle) *  sigmaz(hi, 0) @  sigmaz(hi, 1)
    h +=  np.sin(angle)**2 *  sigmax(hi, 0) @  sigmaz(hi, 1)
    h +=  np.cos(angle) * np.sin(angle) *  sigmax(hi, 0) @  sigmax(hi, 1)
    
    # Last set of terms
    h -=  np.cos(angle)**2 *  sigmax(hi, L-2) @  sigmax(hi, L-1)
    h -=  np.cos(angle) * np.sin(angle) *  sigmax(hi, L-2) @  sigmaz(hi, L-1)
    h -=  np.sin(angle)**2 *  sigmaz(hi, L-2) @  sigmaz(hi, L-1)
    h -=  np.cos(angle) * np.sin(angle) *  sigmaz(hi, L-2) @  sigmax(hi, L-1)
    
    # Middle set of terms (for j = 1 to N-3)
    for j in range(1, L-2):
        h -=  np.cos(angle)**3 *  sigmax(hi, j-1) @  sigmaz(hi, j) @  sigmax(hi, j+1)
        h -=  np.cos(angle)**2 * np.sin(angle) *  sigmaz(hi, j-1) @  sigmaz(hi, j) @  sigmax(hi, j+1)
        h +=  np.cos(angle)**2 * np.sin(angle) *  sigmax(hi, j-1) @  sigmax(hi, j) @  sigmax(hi, j+1)
        h -=  np.cos(angle)**2 * np.sin(angle) *  sigmax(hi, j-1) @  sigmaz(hi, j) @  sigmaz(hi, j+1)
        h +=  np.cos(angle) * np.sin(angle)**2 *  sigmaz(hi, j-1) @  sigmax(hi, j) @  sigmax(hi, j+1)
        h +=  np.cos(angle) * np.sin(angle)**2 *  sigmax(hi, j-1) @  sigmax(hi, j) @  sigmaz(hi, j+1)
        h -=  np.cos(angle) * np.sin(angle)**2 *  sigmaz(hi, j-1) @  sigmaz(hi, j) @  sigmaz(hi, j+1)
        h +=  np.sin(angle)**3 *  sigmaz(hi, j-1) @  sigmax(hi, j) @  sigmaz(hi, j+1)
    j += 1

    h -=  np.cos(angle)**3 *  sigmax(hi, j-1) @  sigmaz(hi, j) @  sigmaz(hi, j+1)
    h -=  np.cos(angle)**2 * np.sin(angle) *  sigmaz(hi, j-1) @  sigmaz(hi, j) @  sigmaz(hi, j+1)
    h +=  np.cos(angle)**2 * np.sin(angle) *  sigmax(hi, j-1) @  sigmax(hi, j) @  sigmaz(hi, j+1)
    h +=  np.cos(angle)**2 * np.sin(angle) *  sigmax(hi, j-1) @  sigmaz(hi, j) @  sigmax(hi, j+1)
    h +=  np.cos(angle) * np.sin(angle)**2 *  sigmaz(hi, j-1) @  sigmax(hi, j) @  sigmaz(hi, j+1)
    h +=  np.cos(angle) * np.sin(angle)**2 *  sigmaz(hi, j-1) @  sigmaz(hi, j) @  sigmax(hi, j+1)
    h -=  np.cos(angle) * np.sin(angle)**2 *  sigmax(hi, j-1) @  sigmax(hi, j) @  sigmax(hi, j+1)
    h -=  np.sin(angle)**3 * sigmaz(hi, j-1) @  sigmax(hi, j) @  sigmax(hi, j+1)
# Save or use the h

    H_ = h.to_sparse()
    eig_vals, eig_vecs = eigsh(H_, k=1, which="SA")
    amp = np.zeros(2**L)
    con = np.zeros(L)
    for i in range(L):
        con[i] = 2**(L-1-i)
    for config in itertools.product([0, 1], repeat=L):
        # Convert to a NumPy array if needed
        config_array = np.array(config)
        p = int(np.sum(con*np.array(config)))
        vec_init = M0[config_array[0]]
        for i in range(1, L-1):
            vec_init = M[config_array[i], :, :, i-1] @ vec_init
        amp[p] = Mlast[config_array[-1]] @ vec_init  
    print(amp @ (H_ @ amp) / (amp @ amp))
    a += 1

0.0
-6.0
0.157
-5.99999994926909
0.314
-5.9999997970763665
0.471
-5.999999543421832
0.628
-5.999999188305496
0.785
-5.999998731727377
0.942
-5.999998173687488
1.1
-5.9999984330457155
1.257
-5.999998946204862
1.414
-5.999999357902234
1.571
-5.9999996681378125


In [256]:
"%.1f" % np.pi

'3.1'

In [238]:
jnp.linspace(0.0, jnp.pi/2, 11)

Array([0.        , 0.15707963, 0.31415927, 0.4712389 , 0.62831853,
       0.78539816, 0.9424778 , 1.09955743, 1.25663706, 1.41371669,
       1.57079633], dtype=float64)

In [234]:
print(amp)

[-1.76776695e-01  1.38777878e-17  2.77555756e-17  1.76776695e-01
 -1.76776695e-01  1.38777878e-17 -2.77555756e-17 -1.76776695e-01
 -1.76776695e-01  2.77555756e-17  2.77555756e-17  1.76776695e-01
  1.76776695e-01 -2.77555756e-17  2.77555756e-17  1.76776695e-01
 -1.76776695e-01  1.38777878e-17  4.16333634e-17  1.76776695e-01
 -1.76776695e-01  2.77555756e-17 -2.77555756e-17 -1.76776695e-01
  1.76776695e-01 -1.38777878e-17 -1.38777878e-17 -1.76776695e-01
 -1.76776695e-01  2.77555756e-17 -4.16333634e-17 -1.76776695e-01
  1.76776695e-01 -2.77555756e-17 -1.38777878e-17 -1.76776695e-01
  1.76776695e-01 -2.77555756e-17  2.77555756e-17  1.76776695e-01
  1.76776695e-01 -1.38777878e-17 -2.77555756e-17 -1.76776695e-01
 -1.76776695e-01  4.16333634e-17 -2.77555756e-17 -1.76776695e-01
 -1.76776695e-01  1.38777878e-17  2.77555756e-17  1.76776695e-01
 -1.76776695e-01  1.38777878e-17 -4.16333634e-17 -1.76776695e-01
  1.76776695e-01 -1.38777878e-17 -1.38777878e-17 -1.76776695e-01
 -1.76776695e-01  2.77555

In [235]:
print(eig_vecs.ravel(), eig_vals)

[-1.76776695e-01  0.00000000e+00 -1.11022302e-16  1.76776695e-01
 -1.76776695e-01  2.77555756e-17  2.77555756e-17 -1.76776695e-01
 -1.76776695e-01  1.24900090e-16 -5.37764278e-17  1.76776695e-01
  1.76776695e-01 -2.77555756e-17 -9.02056208e-17  1.76776695e-01
 -1.76776695e-01  1.11022302e-16 -5.55111512e-17  1.76776695e-01
 -1.76776695e-01  4.33680869e-17  1.11022302e-16 -1.76776695e-01
  1.76776695e-01 -5.55111512e-17  6.93889390e-17 -1.76776695e-01
 -1.76776695e-01  5.55111512e-17  2.77555756e-17 -1.76776695e-01
  1.76776695e-01 -8.32667268e-17  0.00000000e+00 -1.76776695e-01
  1.76776695e-01  2.77555756e-17 -2.77555756e-17  1.76776695e-01
  1.76776695e-01 -2.77555756e-17  9.71445147e-17 -1.76776695e-01
 -1.76776695e-01  1.11022302e-16  5.55111512e-17 -1.76776695e-01
 -1.76776695e-01  1.04083409e-17 -5.55111512e-17  1.76776695e-01
 -1.76776695e-01  1.38777878e-16  0.00000000e+00 -1.76776695e-01
  1.76776695e-01 -8.32667268e-17 -2.77555756e-17 -1.76776695e-01
 -1.76776695e-01 -2.77555

In [117]:
ampx = M_.ravel()
print(ampx @ (H_ @ ampx) / (jnp.dot(ampx , ampx)))


0.16197117849306591


In [6]:
import itertools
L = 4 
for config in itertools.product([0, 1], repeat=L):
    print(config)

(0, 0, 0, 0)
(0, 0, 0, 1)
(0, 0, 1, 0)
(0, 0, 1, 1)
(0, 1, 0, 0)
(0, 1, 0, 1)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 0, 0, 0)
(1, 0, 0, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 0, 0)
(1, 1, 0, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)


In [31]:
print(matrix_list[0])

[[-7.07106781e-01 -7.21644966e-16]
 [-7.21644966e-16  7.07106781e-01]]


In [29]:
matrix = np.load("mps_tensors/tensor_2.npy")
print(matrix)
print(np.linalg.eigvals(matrix[0]))
print(np.linalg.eigvals(matrix[1]))
print(np.linalg.eigv(matrix))

[[[ 0.62239202 -0.33560122]
  [-0.33560122 -0.62239202]]

 [[ 0.62239202 -0.33560122]
  [ 0.33560122  0.62239202]]]
[ 0.70710678 -0.70710678]
[0.62239202+0.33560122j 0.62239202-0.33560122j]
(array([[ 0.70710678+0.j        , -0.70710678+0.j        ],
       [ 0.62239202+0.33560122j,  0.62239202-0.33560122j]]), array([[[ 9.69586312e-01+0.j        ,  2.44749636e-01+0.j        ],
        [-2.44749636e-01+0.j        ,  9.69586312e-01+0.j        ]],

       [[-1.84243051e-16+0.70710678j, -1.84243051e-16-0.70710678j],
        [ 7.07106781e-01+0.j        ,  7.07106781e-01-0.j        ]]]))


In [35]:
import jax
import jax.numpy as jnp
jnp.linalg.norm(jnp.ones((3, 4)), axis = 1)

Array([2., 2., 2.], dtype=float32)

In [55]:
jnp.repeat(jnp.arange(3), 3, axis = 0)

Array([0, 0, 0, 1, 1, 1, 2, 2, 2], dtype=int32)

In [50]:
jnp.angle(jnp.array([3+4*1j, 2+3*1j]))

Array([0.9272952 , 0.98279375], dtype=float32)

In [59]:
import itertools
def generate_combinations(length):
    set1 = [1] * length
    set2 = [3] * length
    combinations = list(itertools.product(*zip(set1, set2)))
    return jnp.array(combinations)
jnp.repeat(generate_combinations(2), 2, axis = 0)

Array([[1, 1],
       [1, 1],
       [1, 3],
       [1, 3],
       [3, 1],
       [3, 1],
       [3, 3],
       [3, 3]], dtype=int32)

In [52]:
jnp.zeros(3, dtype=complex)

Array([0.+0.j, 0.+0.j, 0.+0.j], dtype=complex64)